In [2]:
import pandas as pd
from datetime import datetime

rawpath = "../raw/"

In [3]:
df = pd.read_excel(f"{rawpath}/2022-ES-Tabelle.xlsx", names=['signatur','jahr','datum','medienart','sprache','land','koerperschaft_1','koerperschaft_2','person', 'titel','urheber_vorlage','ort','ort_idn','umfang','bemerkung','vorh'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df.head()

/home/dbsm/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,signatur,jahr,datum,medienart,sprache,land,koerperschaft_1,koerperschaft_2,person,titel,urheber_vorlage,ort,ort_idn,umfang,bemerkung,vorh
0,Bö-GR/V/180,1900,01.04.1900,druck,ger,XA-DXDE,Verlag für Literatur und Kunst,<NA>,"Kufittich, Hans",Errichtung einer Verlagsbuchhandlung.,"Hans Kufittich, Preuß. Hofbuchhändler",Berlin,!040057283!,3 Exemplare,Verlagssignet im Briefkopf,ja
1,Bö-GR/V/356,1900,12.05.1900,druck,ger,XA-DXDE,Deutsche Verlags-Anstalt,<NA>,<NA>,Der buchhändlerische Direktor Emil Büchner tri...,Deutsche Verlags-Anstalt. Der Aufsichtsrat.,Stuttgart,!040582825!,2 Exemplare,Exemplar a mit eigenhändiger Unterschrift von ...,<NA>
2,Bö-GR/V/657,1900,01.06.1900,druck,ger,XA-DXDE,F. Volckmar,C. F. Amelangs Verlag,<NA>,"Hans Volckmar, Sohn des verstorbenen Otto Volc...",Alfred Voerster; Johannes Ziegler,Leipzig,!040352064!,6 Exemplare,Exemplar a mit eigenhändiger Unterschrift der ...,<NA>
3,Bö-GR/V/140,1900,20.06.1900,druck,ger,XA-DXDE,Katholische Vereinsbuchhandlung,Jakob Lutz,"Lutz, Jacob",Gründung eines Verlages und Sortiments einschl...,Katholische Vereinsbuchhandlung,Stuttgart,!040582825!,6 Exemplare,<NA>,ja
4,Bö-GR/V/141,1900,01.07.1900,druck,ger,XA-CH,Christliche Vereinsbuchhandlung,<NA>,<NA>,Rücktritt des Direktors E. C. Schmidtmann. Ern...,Christliche Vereinsbuchhandlung,Zürich,!04068038X!,2 Exemplare,<NA>,ja


In [12]:
def pica_schreiben(row):

    pica = f"""0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 {row.jahr}
"""

    if pd.notna(row.datum):
        d2 = row.datum.split('.')

        if d2[0] == 'XX':
            if (d2[1]) == '01' or (d2[1]) == '03' or (d2[1]) == '05' or (d2[1]) == '07' or (d2[1]) == '08' or (d2[1]) == '10' or (d2[1]) == '12':
                pica += f"1110 $a01.{d2[1]}.{d2[2]}$b31.{d2[1]}.{d2[2]}$n{row.datum}$4ezth\n"
            elif (d2[1]) == '04' or (d2[1]) == '06' or (d2[1]) == '09' or (d2[1]) == '11':
                pica += f"1110 $a01.{d2[1]}.{d2[2]}$b30.{d2[1]}.{d2[2]}$n{row.datum}$4ezth\n"
            elif (d2[1]) == '02':
                pica += f"1110 $a01.{d2[1]}.{d2[2]}$b28.{d2[1]}.{d2[2]}$n{row.datum}$4ezth\n"
            elif (d2[1]) == 'XX':
                pica += f"1110 $a01.01.{d2[2]}$b31.12.{d2[2]}$n{row.datum}$4ezth\n"
        else:
            pica += f"1110 $a{row.datum}$b{row.datum}$n{row.datum}$4ezth\n"

    pica +=f"""7100 {row.signatur}
\t\n"""

    return pica


In [13]:
with open(f"../dat/1110sample.dat", 'w') as f:
    for index, row in df.iterrows():
        f.write(pica_schreiben(row))